# 1- Importing the Hot 100 Billboard List

In [20]:
import pandas as pd
import numpy as np

from sklearn import cluster
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import DBSCAN

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

#Evaluation metrics
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score

from thefuzz import fuzz, process 
import random
import pickle

import config
from config import CLIENT_ID, CLIENT_SECRET

from IPython.display import IFrame

import spotipy 
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [21]:
billboard_df = pd.read_csv("C:\\Users\\ramya\\OneDrive\\Documents\\GitHub\\Song--Recommender\\billboard_df.csv", index_col=0)
billboard_df

,Song,Artist
0,A Bar Song (Tipsy),Shaboozey
1,I Had Some Help,Post Malone Featuring Morgan Wallen
2,Not Like Us,Kendrick Lamar
3,Espresso,Sabrina Carpenter
4,Birds Of A Feather,Billie Eilish
...,...,...
95,Alibi,"Sevdaliza, Pabllo Vittar & Yseult"
96,Residuals,Chris Brown
97,Wine Into Whiskey,Tucker Wetmore
98,Okay,JT


# 2- Importing the Audio Features Dataframe

In [22]:
audio_features_df = pd.read_csv("C:\\Users\\ramya\\OneDrive\\Documents\\GitHub\\Song--Recommender\\total_af_df.csv", index_col=0)
audio_features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.899,0.713,6,-2.752,0,0.3660,0.2550,0.000000,0.0708,0.777,90.051,audio_features,7iL6o9tox1zgHpKUfh9vuC,spotify:track:7iL6o9tox1zgHpKUfh9vuC,https://api.spotify.com/v1/tracks/7iL6o9tox1zg...,https://api.spotify.com/v1/audio-analysis/7iL6...,193467,4
1,0.489,0.692,1,-6.672,1,0.4100,0.0322,0.006080,0.3150,0.527,96.946,audio_features,4nva9EpKntUTs6CRSGBCn9,spotify:track:4nva9EpKntUTs6CRSGBCn9,https://api.spotify.com/v1/tracks/4nva9EpKntUT...,https://api.spotify.com/v1/audio-analysis/4nva...,237707,4
2,0.613,0.574,11,-7.961,1,0.4670,0.0253,0.000031,0.3800,0.755,125.180,audio_features,5D2mYZuzcgjpchVY1pmTPh,spotify:track:5D2mYZuzcgjpchVY1pmTPh,https://api.spotify.com/v1/tracks/5D2mYZuzcgjp...,https://api.spotify.com/v1/audio-analysis/5D2m...,209107,5
3,0.589,0.827,11,-2.938,1,0.3840,0.0143,0.001510,0.1400,0.386,161.978,audio_features,5STdMlrBf6pqWiNE7WqxSi,spotify:track:5STdMlrBf6pqWiNE7WqxSi,https://api.spotify.com/v1/tracks/5STdMlrBf6pq...,https://api.spotify.com/v1/audio-analysis/5STd...,256240,4
4,0.886,0.626,11,-2.528,0,0.2770,0.2650,0.000000,0.0240,0.856,93.989,audio_features,6mcu7D7QuABVwUGDwovOEh,spotify:track:6mcu7D7QuABVwUGDwovOEh,https://api.spotify.com/v1/tracks/6mcu7D7QuABV...,https://api.spotify.com/v1/audio-analysis/6mcu...,196627,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12824,0.331,0.548,4,-5.685,1,0.0286,0.2860,0.000000,0.2510,0.216,135.263,audio_features,66RwIlYInGwQB4BokNhRnd,spotify:track:66RwIlYInGwQB4BokNhRnd,https://api.spotify.com/v1/tracks/66RwIlYInGwQ...,https://api.spotify.com/v1/audio-analysis/66Rw...,260000,4
12825,0.388,0.177,7,-17.819,1,0.0325,0.6830,0.000009,0.2610,0.106,124.444,audio_features,72NcOvvGdarZaOyKMNHZ1m,spotify:track:72NcOvvGdarZaOyKMNHZ1m,https://api.spotify.com/v1/tracks/72NcOvvGdarZ...,https://api.spotify.com/v1/audio-analysis/72Nc...,228093,4
12826,0.698,0.298,9,-15.192,0,0.0379,0.5110,0.008470,0.1080,0.260,117.459,audio_features,6wx2cwyutp7cgzyi3EYVnM,spotify:track:6wx2cwyutp7cgzyi3EYVnM,https://api.spotify.com/v1/tracks/6wx2cwyutp7c...,https://api.spotify.com/v1/audio-analysis/6wx2...,241200,4
12827,0.311,0.606,8,-8.264,1,0.0702,0.6300,0.000000,0.9840,0.265,112.029,audio_features,7FHmQOpMo25rPnZeUmCLEQ,spotify:track:7FHmQOpMo25rPnZeUmCLEQ,https://api.spotify.com/v1/tracks/7FHmQOpMo25r...,https://api.spotify.com/v1/audio-analysis/7FHm...,426227,4


# 3- Imporing the Audio Features df, the Scaler & the Model  

In [23]:
df_final = pd.read_csv("C:\\Users\\ramya\\OneDrive\\Documents\\GitHub\\Song--Recommender\\df_final.csv", index_col = [0])
df_final

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,KMeans_Cluster
0,7iL6o9tox1zgHpKUfh9vuC,0.918284,0.713921,0.545455,0.877186,0.0,0.399128,0.256023,0.000000,0.071443,0.789634,0.427110,0.104262,0.8,4
1,4nva9EpKntUTs6CRSGBCn9,0.499489,0.692842,0.090909,0.784674,1.0,0.447110,0.032328,0.006111,0.317861,0.535569,0.459813,0.128928,0.8,6
2,5D2mYZuzcgjpchVY1pmTPh,0.626149,0.574395,1.000000,0.754254,1.0,0.509269,0.025400,0.000031,0.383451,0.767276,0.593726,0.112982,1.0,8
3,5STdMlrBf6pqWiNE7WqxSi,0.601634,0.828353,1.000000,0.872796,1.0,0.418757,0.014356,0.001518,0.141271,0.392276,0.768258,0.139261,0.8,15
4,6mcu7D7QuABVwUGDwovOEh,0.905005,0.626592,1.000000,0.882472,0.0,0.302072,0.266063,0.000000,0.024218,0.869919,0.445788,0.106024,0.8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11112,66RwIlYInGwQB4BokNhRnd,0.338100,0.548297,0.363636,0.807967,1.0,0.031189,0.287148,0.000000,0.253280,0.219512,0.641549,0.141358,0.8,6
11113,72NcOvvGdarZaOyKMNHZ1m,0.396323,0.175893,0.636364,0.521606,1.0,0.035442,0.685743,0.000009,0.263370,0.107724,0.590235,0.123568,0.8,3
11114,6wx2cwyutp7cgzyi3EYVnM,0.712972,0.297351,0.818182,0.583603,0.0,0.041330,0.513052,0.008513,0.108981,0.264228,0.557105,0.130876,0.8,16
11115,7FHmQOpMo25rPnZeUmCLEQ,0.317671,0.606517,0.727273,0.747103,1.0,0.076554,0.632530,0.000000,0.992936,0.269309,0.531351,0.234038,0.8,3


In [24]:
with open('Pickle_Files/kmeans_19.pkl', 'rb') as file:
    kmeans_19 = pickle.load(file)

C:\Users\ramya\anaconda3\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KMeans from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [25]:
with open('Pickle_Files/minmax.pkl', 'rb') as file:
    minmax = pickle.load(file)

C:\Users\ramya\anaconda3\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# 4- Important Functions

In [26]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [27]:
import requests
import base64

def get_spotify_token(CLIENT_ID, CLIENT_SECRET):

    client_credentials = f"{CLIENT_ID}:{ CLIENT_SECRET}"
    client_credentials_b64 = base64.b64encode(client_credentials.encode()).decode()

    headers = {
        "Authorization": f"Basic {client_credentials_b64}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }

    response = requests.post("https://accounts.spotify.com/api/token", headers=headers, data=data)

    if response.status_code == 200:
        token = response.json().get("access_token")
        return token
    else:
        raise Exception(f"Failed to get token: {response.status_code} {response.text}")

client_id = CLIENT_ID
client_secret = CLIENT_SECRET
token = get_spotify_token(client_id, client_secret)
print(f"Your Spotify OAuth token is: {token}")


Your Spotify OAuth token is: BQAxLknpoc1H3t0bZzAWnxtbAS7nnLdDMBKm89_tPhu3cssQe8Ke8GC7epvcfH2puCaGa96AszJ0y8S3swqvr6NwXWA9lnbgsduc3lxuzGpoI2JNS-I


In [28]:
def get_song_name(recommended_song_id):
    headers = {
        "Authorization": f"Bearer {token}"
    }
    url = f"https://api.spotify.com/v1/tracks/{recommended_song_id}"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        return data['name']
    else:
        return "Error: Unable to retrieve song name"

In [29]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.CLIENT_ID,
                                                           client_secret= config.CLIENT_SECRET))

# 5- The program

In [36]:
def recommend_a_song(song_pref):
    song_ref_names = []
    song_ref_ids = []
    song_ref_artist = []

    song_matching_percent = process.extractOne(song_pref.lower(), billboard_df['Song'])[1]
    song_name = process.extractOne(song_pref.lower(), billboard_df['Song'])[0]
    artist_name = billboard_df.loc[billboard_df["Song"] == song_name, 'Artist'].values[0] 
    
    if song_matching_percent > 80:
        while True:
            question = input(f"is this song ({song_name}) by ({artist_name}) the song you mean? (Y,N) ").lower().strip()
            if question in ["y","n"]:
                break
            else:
                print("Please answer with (Y) or (N)")
        if question == "y":
            updated_billboard_df = billboard_df.drop(billboard_df[billboard_df['Song'] == song_name].index)
            recommended_song = random.choice(updated_billboard_df['Song'])
            artist_of_recommended_song = updated_billboard_df.loc[updated_billboard_df["Song"] == recommended_song, 'Artist'].values[0]                                 
            print(f"We can recommend you this song: ({recommended_song}) by ({artist_of_recommended_song})")
            recommended_billboard_song_id = sp.search(q=recommended_song,limit=1,market="GB")["tracks"]["items"][0]["id"]
            return play_song(recommended_billboard_song_id)

        elif question == "n":
            print("Sorry, you are not cool enough to have your song in the Top 100 Billboard songs!")
            print("We will recommend you another song")

            for i in sp.search(q=song_pref, limit=10)["tracks"]["items"][0:10]:
                song_ref_names.append(i['name'])
                song_ref_ids.append(i['id'])
                song_ref_artist.append(i['artists'][0]['name'])

            for name, track_id, artist in zip(song_ref_names, song_ref_ids, song_ref_artist):
                while True:
                    question_2 = input(f"Is this song ({name}) by ({artist}) the song you mean? (Y,N) ").lower().strip()
                    if question_2 in ["y", "n"]:
                        break
                    else:
                        print("Please answer with (Y) or (N)")

                if question_2 == "y":
                    song_ref_af = sp.audio_features(track_id)[0]
                    song_ref_af = pd.DataFrame(song_ref_af, index=[0])
                    
                    #

                    song_ref_af.drop(columns=['type', 'uri', 'track_href', 'analysis_url', 'id'], inplace=True)
                    song_ref_af_scaled = pd.DataFrame(minmax.transform(song_ref_af), columns=song_ref_af.columns)
                    song_ref_cluster = kmeans_19.predict(song_ref_af_scaled)[0]
                    #df_final = pd.concat([id_column, df], axis=1)
                    recommended_song_id = df_final[df_final['KMeans_Cluster'] == song_ref_cluster].sample()['id'].values[0]
                    token = get_spotify_token(CLIENT_ID, CLIENT_SECRET)
                    recommended_song_name = get_song_name(recommended_song_id)
                    print(f"We recommend you this song: {recommended_song_name}")
                    return play_song(recommended_song_id)
                    break

                elif question_2 == "n":
                    print("Sorry, I will show you another song!")
                    continue
    else:
        print("Sorry, you are not cool enough to have your song in the Top 100 Billboard songs!")
        print("We will recommend you another song")

        for i in sp.search(q=song_pref, limit=10)["tracks"]["items"][0:10]:
            song_ref_names.append(i['name'])
            song_ref_ids.append(i['id'])
            song_ref_artist.append(i['artists'][0]['name'])

        for name, track_id, artist in zip(song_ref_names, song_ref_ids, song_ref_artist):
            while True:
                question_2 = input(f"Is this song ({name}) by ({artist}) the song you mean? (Y,N) ").lower().strip()
                if question_2 in ["y", "n"]:
                    break
                else:
                    print("Please answer with (Y) or (N)")

            if question_2 == "y":
                song_ref_af = sp.audio_features(track_id)[0]
                song_ref_af = pd.DataFrame(song_ref_af, index=[0])
                
                #
                
                song_ref_af.drop(columns=['type', 'uri', 'track_href', 'analysis_url', 'id'], inplace=True)
                song_ref_af_scaled = pd.DataFrame(minmax.transform(song_ref_af), columns=song_ref_af.columns)
                song_ref_cluster = kmeans_19.predict(song_ref_af_scaled)[0]
                #df_final = pd.concat([id_column, df], axis=1)
                recommended_song_id = df_final[df_final['KMeans_Cluster'] == song_ref_cluster].sample()['id'].values[0]
                token = get_spotify_token(CLIENT_ID, CLIENT_SECRET)
                recommended_song_name = get_song_name(recommended_song_id)
                print(f"We recommend you this song: {recommended_song_name}")
                return play_song(recommended_song_id)
                break  

            elif question_2 == "n":
                print("Sorry, I will show you another song!")
                continue
    print("We couldn't find the song you like, please run this cell again with the right name of the song")

In [38]:
recommend_a_song("despacito")

Sorry, you are not cool enough to have your song in the Top 100 Billboard songs!
We will recommend you another song
Is this song (Despacito) by (Luis Fonsi) the song you mean? (Y,N) n
Sorry, I will show you another song!
Is this song (Despacito - Remix) by (Luis Fonsi) the song you mean? (Y,N) y
We recommend you this song: Error: Unable to retrieve song name


In [ ]:
# play the user song